In [5]:
#библиотека интерфейса
from tkinter import *
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
#библиотека для ввода даты
from tkcalendar import DateEntry
#библиотека для экселя
import openpyxl
from openpyxl.utils import get_column_letter
import requests
from openpyxl.utils.exceptions import CellCoordinatesException
import re
import openpyxl.cell.cell
#библиотеки для графиков
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from math import *



wb = openpyxl.load_workbook('20221.xlsx') #вызов таблиц и листов
sheet_2022 = wb['2022']


def input_expense_in_cells():
    for date_cell in sheet_2022['A']:
        date_column = 1
        if date_cell.value is None:
            date_column = date_cell.row
            break
        else:
            date_column = date_cell.row + 1
    sheet_2022['A'+str(date_column)] = cal.get()
    sheet_2022['B'+str(date_column)] = str(variable.get())
    sheet_2022['C'+str(date_column)] = float(expence_input.get())
    sheet_2022['D'+str(date_column)] = str(desc_input.get())
    wb.save('20221.xlsx')

def count_sum_month():
    summonth = 0
    for i in range(1,len(sheet_2022['C'])+1):
        if isinstance(sheet_2022.cell(row = i, column = 3).value,(float, int)): # проверка на тип int/float
            summonth+= sheet_2022.cell(row = i, column = 3).value # Суммируем числа в ячейках третьего столбца
    sheet_2022['E2'].value = summonth
    text_sum = round(summonth)
    lbl_stat.config(text = text_sum)
    wb.save('20221.xlsx')
    
def input_monthsum_in_cell():
    monthsum = float(month_sum.get())
    sheet_2022['F2'].value = monthsum
    wb.save('20221.xlsx')

def count_balance_month():
    monthsum= sheet_2022['F2'].value
    summonth = sheet_2022['E2'].value
    balance = monthsum - summonth
    sheet_2022['G2'].value = balance
    text_balance = round(balance)
    lbl_statr.config(text=text_balance)
    wb.save('20221.xlsx')
    
def check_values():
    try:
        float(expence_input.get())
        input_expense_in_cells()
    except ValueError:
        create_errorwindow()

def create_errorwindow():
    window = tk.Toplevel(tracker)
    window.title ('Ошибка')
    errorlabel=Label(window, text = 'Введи число!', font='Arial 12')
    errorlabel.place(x=50,y=10)
    errorbutton = Button(window, text = 'OK', width = 20, command = window.destroy)
    errorbutton.place(x=20,y=50)

def count_stats():
    df = pd.read_excel('20221.xlsx') 
    figure1 = plt.Figure(figsize=(7,6), dpi=50)
    ax1 = figure1.add_subplot(111)
    bar1 = FigureCanvasTkAgg(figure1, tracker)
    bar1.get_tk_widget().place(x=450, y=190)
    df = df[['Дата','Сумма']].groupby('Дата').sum()
    df.plot(kind='bar', legend=True, ax=ax1)
    ax1.set_title('Ежедневные траты')
     
 

#вызов интерфейса
tracker = Tk()
tracker.title ('Трекер расходов')
tracker.geometry('900x550')



#объявление элементов
lbl = Label(tracker, text = 'Трекер расходов', font = 'Arial 18')
lbl.place(x=350,y=1)

lbl_date = Label(tracker, text = 'Дата:', font = 'Arial 12')
lbl_date.place(x=40,y=50)
cal=DateEntry(tracker,selectmode='day',width=30)
cal.place(x=40,y=80)

lbl_expense = Label(tracker, text= 'Сумма:', font = 'Arial 12')
lbl_expense.place(x=250, y = 50)
expence_input = Entry(tracker, width=30)
expence_input.place(x=250,y=80)

lbl_category = Label(tracker, text = 'Категория:', font = 'Arial 12')
lbl_category.place(x=450, y = 50)
OptionList = ['Постоянные', 'Питание', 'Продукты', 'Рестораны', 'Проезд', 'Развлечения', 'Покупки', 'Прочее']
variable = StringVar()
variable.set(OptionList[0])
categories_expence = OptionMenu(tracker, variable, *OptionList)
categories_expence.place(x=450,y=75)
categories_expence.config (width=25)

lbl_desc = Label(tracker, text = 'Описание:', font = 'Arial 12')
lbl_desc.place(x=660,y=50)
desc_input = Entry(tracker, width=30)
desc_input.place(x=660, y=80)

but = Button(tracker, text = 'Ввести', width = 30, command = check_values)
but.place(x=350,y=120)

framestats = LabelFrame(tracker, text='Статистика', heigh = 350, width = 810)
framestats.place(x=40, y = 170)

update_stat = Button(tracker, text = 'Обновить статистику', command = lambda:[count_sum_month(), count_balance_month(), count_stats()])
update_stat.place(x=50,y=270)

label_sum = Label(tracker, text='Сумма за месяц', font='Arial 12')
label_sum.place(x=50,y=200)
lbl_stat = Label(tracker, text = '0', font= 'Arial 12')
lbl_stat.place(x=200,y=200)

label_reminder = Label(tracker, text='Остаток', font='Arial 12')
label_reminder.place(x=50,y=230)
lbl_statr = Label(tracker, text = '0', font= 'Arial 12')
lbl_statr.place(x=200,y=230)

month_sum = Entry(tracker, width = 30)
month_sum.place(x=50, y=480)
month_sum_button=Button(tracker, text = 'Изменить зп',command = input_monthsum_in_cell)
month_sum_button.place(x=250,y=475)

df = pd.read_excel('20221.xlsx') 
figure1 = plt.Figure(figsize=(7,6), dpi=50)
ax1 = figure1.add_subplot(111)
bar1 = FigureCanvasTkAgg(figure1, tracker)
bar1.get_tk_widget().place(x=450, y=190)
df = df[['Дата','Сумма']].groupby('Дата').sum()
df.plot(kind='bar', legend=True, ax=ax1)
ax1.set_title('Ежедневные траты')

#вывод окна
tracker.mainloop ()